# IE402 Assignment 2
### By: 201801015
### 11-10-2020
##### The first execution is for the problem given in the assignment
##### and the later executions are problems from the internet (verified results)
##### call simplex() function

In [34]:
def simplex():
    import numpy as np  
    from fractions import Fraction 
    from math import nan

    # inputs
    n, m = input("Enter number of basis variables and number of constraints seperated by a space:").split(" ")
    n, m = int(n), int(m)
    
    # Coefficients of objective function Z
    c = input("Enter the coefficients of objective function seperated by a space:").split(" ")
    c = np.array(c, dtype = 'int')
    c = np.append(c, np.zeros(m))
    Cj = np.array(np.append(["" ,"", "", "Cj"],c))
    
    # constraints parameters input
    identity = np.identity(m)
    A = np.array([])
    for i in range(m):
        x = input("Enter constraint " + str(i+1) + " parameters seperated by a space:").split(" ")
        A = np.append(A,x)
    A = A.reshape(m,n+2)
    
    # append identity to A
    A = np.c_[A,identity]
    
    # remove leq from matrix
    A = np.delete(A,n,axis = 1)
    
    # remove constraint limit values to get the constraint coefficients array
    b = A[:,n]
    A = np.delete(A,n,axis = 1)
        
    #initial basis variables indexed from 0 consisting of slack variables
    #cb = np.array(c[n+m-1]) 
    B = [i+n for i in range(m)]
    B = np.array(B, dtype ='float')
    B = B.reshape(m,1)  
    
    # cb contains their corresponding coefficients in Z    
    cb = np.zeros(m)   
    cb = cb.reshape(m,1)
    xb = np.transpose([b])
    MR = np.zeros(m)
    MR = MR.reshape(m,1)
    #OP = np.zeros(m)
    #OP = OP.reshape(m,1)
    OP = np.array(["                         " for _ in range(m)])
    
    # combine matrices B and cb 
    table = np.hstack((B, cb))              
    table = np.hstack((table, xb))  
    
    # combine matrices B, cb and xb 
    # finally combine matrix A to form the complete simplex table 
    table = np.hstack((table, A))        
    table = np.hstack((table, MR))  
    #table = np.hstack((table, OP)) 
    
    # change the type of table to float 
    table = np.array(table, dtype ='float')  
    # inputs end 

    # if min problem, make this var 1 
    MIN = 0
    
    print('\033[1m'+"\n\nSimplex Algorithm: Tabular Method\n\n") 

    # optimal => reached = 1
    reached = 0  
    # track of number to tables
    itr = 1
    # unbounded LPP
    unbounded = 0
    pivot = 0
    
    # loop till optimal
    while reached == 0: 

        # calculate Relative profits-> zj - cj 
        i = 0
        sum = 0
        while i<m: 
            sum += c[int(table[i][0])]*table[i][2] 
            i+= 1
        i = 0
        rel_prof = [] 
        
        while i<len(A[0]): 
            rel_prof.append(-c[i] + np.sum(table[:, 1]*table[:, 3 + i])) 
            i = i + 1

        flag = 0
        for profit in rel_prof: 
            if profit<0: 
                flag = 1
                break
            # if all relative profits <= 0 
        if flag == 0: 
            reached = 1

        # kth var will enter the basis 
        newIn = 99999
        for profit in rel_prof: 
            if profit<newIn:
                newIn = profit
        k = rel_prof.index(newIn) 
        minn = 99999
        i = 0; 
        r = -1
        
        # min ratio test (only positive values) 
        while i<len(table): 
            if (table[:, 2][i]>0 and table[:, 3 + k][i]>0):  
                val = table[:, 2][i]/table[:, 3 + k][i] 
                table[:,3+len(c)][i] = val
                if val<minn: 
                    minn = val 
                    r = i     # leaving variable 
            else:
                table[:,3+len(c)][i] = nan
            i+= 1
        
        if reached == 0:
            pivot = table[r][3 + k] 
        
        print("Iteration: ", end =' ') 
        print(itr) 
        
        for ele in Cj:
            print(ele, end="\t")
        print()
        print("CB \tB \tXB \tb", end = '') 
        for i in range(m+n):
            print("\ta"+str(i+1), end='')
        print("\tMR \tO/P")
        # this step is done to change the 0 indexing to 1 indexing for correct display
        table[:, 0] = table[:, 0] + 1
        table[:,[0, 1]] = table[:,[1, 0]]
        ii = 0;
        for row in table:
            if reached == 1:
                for el in range(len(row)-1):
                    if el == 1:
                        print('a'+str(Fraction(str(round(row[el],2)))), end ='\t')
                        print('x'+str(Fraction(str(round(row[el],2)))), end ='\t')
                    else:
                        print(str(round(row[el],3)), end ='\t')  
                print("\t", OP[ii], end="\t")
            else:
                for el in range(len(row)):
                    if el == 1:
                        print('a'+str(Fraction(str(round(row[el],2)))), end ='\t')
                        print('x'+str(Fraction(str(round(row[el])))), end ='\t')
                    else:
                        print(str(round(row[el],3)), end ='\t')
                print(OP[ii], end="\t")
            print() 
            ii+=1
        table[:,[0, 1]] = table[:,[1, 0]]
        # this step is done to undo the 1 indexing to 0 indexing for correct calculations
        table[:, 0] = table[:, 0] - 1

        print("     Zj - Cj ", end ='\t\t') 
        print(Fraction(str(round(sum,2))), end ='\t') 
        for profit in rel_prof: 
            print(Fraction(str(round(profit,2))), end ='\t') 
        print() 
        
        #if reached == 0:
            #pivot = table[r][3 + k] 
            #print("pivot element: ", end =" ") 
            #print(Fraction(pivot+1).limit_denominator(100)) 
        #print() 
        
        b_var = table[:, 0] 
        
        # if no min ratio test was performed 
        if r ==-1: 
            unbounded = 1 
            break

        if flag == 0: 
            print("The optimum solution is:") 
            break

        # perform row operations 
        # find row operations as strings
        i = 0
        while i<m: 
            if i != r: 
                OP[i] = "R"+str(i+1)+"' - "+str(round(table[i][3 + k],2))+"*R"+str(r+1)+"'"
            else:
                OP[i] = "R"+str(r+1)+"'/"+str(round(pivot,2))
            i += 1
        # divide the pivot row with the pivot element 
        table[r, 2:len(table[0])] = table[ 
                r, 2:len(table[0])] / pivot 
        
        # do row operation on other rows 
        i = 0
        while i<len(table): 
            if i != r: 
                table[i, 2:len(table[0])] = table[i, 2:len(table[0])] - table[i][3 + k] * table[r, 2:len(table[0])] 
                #OP[i] = "R"+str(i+1)+" - "+str(table[i][3 + k])+"*R"+str(r+1)
            i += 1

        # assign the new basic variable 
        table[r][0] = k 
        table[r][1] = c[k] 

        #print() 
        itr+= 1

    if unbounded == 1: 
        print("UNBOUNDED LPP") 
        exit() 
        
    basis = [] 
    i = 0
    sum = 0
    while i<m: 
        sum += c[int(table[i][0])]*table[i][2] 
        i+= 1
    
    #print 0 to n-1 basis variables
    for ele in range(n): 
        if ele in table[:,0]:
            print("x"+str(ele+1)+" = "+str(round(table[:,2][np.where(table[:,0] == ele)][0],2)), end=", ")
        else:
            print("x"+str(ele+1)+" = 0", end=", ")
    print("and Z =", end=" ")
    # if MIN problem make z negative 
    if MIN == 1: 
        print(-Fraction(str(sum)).limit_denominator(100)) 
    else: 
        print(Fraction(str(sum)).limit_denominator(100)) 
    
    print()
    print("Simplex Finished Successfully!") 


In [35]:
# call simplex() function
simplex()

Enter number of basis variables and number of constraints seperated by a space:3 2
Enter the coefficients of objective function seperated by a space:1 1 3
Enter constraint 1 parameters seperated by a space:3 2 1 leq 3
Enter constraint 2 parameters seperated by a space:2 1 2 leq 2


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	1.0	1.0	3.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a4	x4	3.0	3.0	2.0	1.0	1.0	0.0	3.0	                         	
0.0	a5	x5	2.0	2.0	1.0	2.0	0.0	1.0	1.0	                         	
     Zj - Cj 		0	-1	-1	-3	0	0	
Iteration:  2
			Cj	1.0	1.0	3.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a4	x4	2.0	2.0	1.5	0.0	1.0	-0.5		 R1' - 1.0*R2'	
3.0	a3	x3	1.0	1.0	0.5	1.0	0.0	0.5		 R2'/2.0	
     Zj - Cj 		3	2	1/2	0	0	3/2	
The optimum solution is:
x1 = 0, x2 = 0, x3 = 1.0, and Z = 3

Simplex Finished Successfully!


In [36]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 3
Enter the coefficients of objective function seperated by a space:3 2
Enter constraint 1 parameters seperated by a space:2 1 leq 18
Enter constraint 2 parameters seperated by a space:2 3 leq 42
Enter constraint 3 parameters seperated by a space:3 1 leq 24


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	3.0	2.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a3	x3	18.0	2.0	1.0	1.0	0.0	0.0	9.0	                         	
0.0	a4	x4	42.0	2.0	3.0	0.0	1.0	0.0	21.0	                         	
0.0	a5	x5	24.0	3.0	1.0	0.0	0.0	1.0	8.0	                         	
     Zj - Cj 		0	-3	-2	0	0	0	
Iteration:  2
			Cj	3.0	2.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a3	x3	2.0	0.0	0.333	1.0	0.0	-0.667	6.0	R1' - 2.0*R3'	
0.0	a4	x4	26.0	0.0	2.333	0.0	1.0	-0.667	11.143	R2' - 2.0*R3'	
3.0	a1	x1	8.0	1.0	0.333	0.0	0.0	0.333	24.0	R3'/3.0	
     Zj - Cj 		24	0	-1	0	0	1	
Iteration:  3
			Cj	3.0	2.0	0.0	0.0	0.

In [37]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 3
Enter the coefficients of objective function seperated by a space:40 30
Enter constraint 1 parameters seperated by a space:1 2 leq 16
Enter constraint 2 parameters seperated by a space:1 1 leq 9
Enter constraint 3 parameters seperated by a space:3 2 leq 24


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	40.0	30.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a3	x3	16.0	1.0	2.0	1.0	0.0	0.0	16.0	                         	
0.0	a4	x4	9.0	1.0	1.0	0.0	1.0	0.0	9.0	                         	
0.0	a5	x5	24.0	3.0	2.0	0.0	0.0	1.0	8.0	                         	
     Zj - Cj 		0	-40	-30	0	0	0	
Iteration:  2
			Cj	40.0	30.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	MR 	O/P
0.0	a3	x3	8.0	0.0	1.333	1.0	0.0	-0.333	6.0	R1' - 1.0*R3'	
0.0	a4	x4	1.0	0.0	0.333	0.0	1.0	-0.333	3.0	R2' - 1.0*R3'	
40.0	a1	x1	8.0	1.0	0.667	0.0	0.0	0.333	12.0	R3'/3.0	
     Zj - Cj 		320	0	-333/100	0	0	1333/100	
Iteration:  3
			Cj	4

In [38]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:3 3
Enter the coefficients of objective function seperated by a space:1 2 -1
Enter constraint 1 parameters seperated by a space:2 1 1 leq 14
Enter constraint 2 parameters seperated by a space:4 3 2 leq 28
Enter constraint 3 parameters seperated by a space:2 5 5 leq 30


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	1.0	2.0	-1.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	a6	MR 	O/P
0.0	a4	x4	14.0	2.0	1.0	1.0	1.0	0.0	0.0	14.0	                         	
0.0	a5	x5	28.0	4.0	3.0	2.0	0.0	1.0	0.0	9.333	                         	
0.0	a6	x6	30.0	2.0	5.0	5.0	0.0	0.0	1.0	6.0	                         	
     Zj - Cj 		0	-1	-2	1	0	0	0	
Iteration:  2
			Cj	1.0	2.0	-1.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	a6	MR 	O/P
0.0	a4	x4	8.0	1.6	0.0	0.0	1.0	0.0	-0.2	5.0	R1' - 1.0*R3'	
0.0	a5	x5	10.0	2.8	0.0	-1.0	0.0	1.0	-0.6	3.571	R2' - 3.0*R3'	
2.0	a2	x2	6.0	0.4	1.0	1.0	0.0	0.0	0.2	15.0	R3'/5.0	
     Zj - Cj 		12	-1/5	0	3	

In [39]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 2
Enter the coefficients of objective function seperated by a space:3 1
Enter constraint 1 parameters seperated by a space:2 1 leq 8
Enter constraint 2 parameters seperated by a space:2 3 leq 12


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	3.0	1.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	MR 	O/P
0.0	a3	x3	8.0	2.0	1.0	1.0	0.0	4.0	                         	
0.0	a4	x4	12.0	2.0	3.0	0.0	1.0	6.0	                         	
     Zj - Cj 		0	-3	-1	0	0	
Iteration:  2
			Cj	3.0	1.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	MR 	O/P
3.0	a1	x1	4.0	1.0	0.5	0.5	0.0		 R1'/2.0	
0.0	a4	x4	4.0	0.0	2.0	-1.0	1.0		 R2' - 2.0*R1'	
     Zj - Cj 		12	0	1/2	3/2	0	
The optimum solution is:
x1 = 4.0, x2 = 0, and Z = 12

Simplex Finished Successfully!


In [40]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 2
Enter the coefficients of objective function seperated by a space:40 30
Enter constraint 1 parameters seperated by a space:1 1 leq 12
Enter constraint 2 parameters seperated by a space:2 1 leq 16


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	40.0	30.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	MR 	O/P
0.0	a3	x3	12.0	1.0	1.0	1.0	0.0	12.0	                         	
0.0	a4	x4	16.0	2.0	1.0	0.0	1.0	8.0	                         	
     Zj - Cj 		0	-40	-30	0	0	
Iteration:  2
			Cj	40.0	30.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	MR 	O/P
0.0	a3	x3	4.0	0.0	0.5	1.0	-0.5	8.0	R1' - 1.0*R2'	
40.0	a1	x1	8.0	1.0	0.5	0.0	0.5	16.0	R2'/2.0	
     Zj - Cj 		320	0	-10	0	20	
Iteration:  3
			Cj	40.0	30.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	MR 	O/P
30.0	a2	x2	8.0	0.0	1.0	2.0	-1.0		 R1'/0.5	
40.0	a1	x1	4.0	1.0	0.0	-1.0	1.0		 R2' - 0.5*R1'	
     Zj - Cj 		400	0	0	20	10	
The optimum solution is:
x1 = 4.0, x2 = 8.0, and Z = 400

Simple

In [42]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:3 3
Enter the coefficients of objective function seperated by a space:2 -1 2
Enter constraint 1 parameters seperated by a space:2 1 0 leq 10
Enter constraint 2 parameters seperated by a space:1 2 -2 leq 10
Enter constraint 3 parameters seperated by a space:0 1 2 leq 5


Simplex Algorithm: Tabular Method


Iteration:  1
			Cj	2.0	-1.0	2.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	a6	MR 	O/P
0.0	a4	x4	10.0	2.0	1.0	0.0	1.0	0.0	0.0	5.0	                         	
0.0	a5	x5	10.0	1.0	2.0	-2.0	0.0	1.0	0.0	10.0	                         	
0.0	a6	x6	5.0	0.0	1.0	2.0	0.0	0.0	1.0	nan	                         	
     Zj - Cj 		0	-2	1	-2	0	0	0	
Iteration:  2
			Cj	2.0	-1.0	2.0	0.0	0.0	0.0	
CB 	B 	XB 	b	a1	a2	a3	a4	a5	a6	MR 	O/P
2.0	a1	x1	5.0	1.0	0.5	0.0	0.5	0.0	0.0	nan	R1'/2.0	
0.0	a5	x5	5.0	0.0	1.5	-2.0	-0.5	1.0	0.0	nan	R2' - 1.0*R1'	
0.0	a6	x6	5.0	0.0	1.0	2.0	0.0	0.0	1.0	2.5	R3' - 0.0*R1'	
     Zj - Cj 		10	0	2	-2	1	0	0	
It

In [ ]:
simplex()